<a href="https://colab.research.google.com/github/kimdarwin/ule/blob/main/alpaca_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Talk to Alpaca-LoRA

This notebook contains minimal code for running [Alpaca-LoRA](https://github.com/tloen/alpaca-lora/) for demonstration purposes. Please check the repo for more details.

In [3]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip install -q git+https://github.com/huggingface/peft.git


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 19.9 MB/s eta 0:00:00
     

In [4]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "tloen/alpaca-lora-7b")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [5]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [6]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def evaluate(instruction, input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    output2 = ""
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        print("Response:", output.split("### Response:")[1].strip())
        output2 = output2+" "+output.split("### Response:")[1].strip()
    return output2.strip()

In [7]:
a =evaluate("Instruction: answer a question based on choices. The answer can be an unpopular or short word.\nBob said, \"The 75th annual Grammy Awards, which honor the best Broadway productions of the season, took place at New York's Radio City Music Hall on Sunday. \"A Strange Loop\" won the award for Best Musical and for Best Book of a Musical.\"\nQ: Broadway's biggest stars were recognized this week at which prestigious awards showin Bob's opinion? Choices: Grammy, The Academy Awards, The Emmy Awards, The Tony Awards, I don't know")
print(str(a))
a =evaluate("Instruction: answer a question based on choices.\nBob said, \"The 75th annual Grammy Awards, which honor the best Broadway productions of the season, took place at New York's Radio City Music Hall on Sunday. \"A Strange Loop\" won the award for Best Musical and for Best Book of a Musical.\"\nQ: Broadway's biggest stars were recognized this week at which prestigious awards showin Bob's opinion? Choices: Grammy, The Academy Awards, The Emmy Awards, The Tony Awards, I don't know")
print(str(a))

Response: The Tony Awards
The Tony Awards
Response: The Tony Awards.
The Tony Awards.


In [8]:
a =evaluate("Instruction: answer a question based on the choices.\nBob said, \"European regulators announced a plan this week that will eliminate the infuriating number of charging cables needed for various devices. Under the legislation, virtually all of your everyday devices that are rechargeable via a wired cable -- phones, tablets, e-readers, earbuds, cameras, portable speakers, etc -- will have to be equipped with a port known as USB-C.\"\nQ: To help reduce cable clutter and waste, which continent will soon require Apple and other smartphone makers to support a single common charging standard for all mobile devicesin Bob's opinion?")
a =evaluate("Instruction: answer a question based on the choices. The answer can be an unpopular or short word.\nBob said, \"Asiaan regulators announced a plan this week that will eliminate the infuriating number of charging cables needed for various devices. Under the legislation, virtually all of your everyday devices that are rechargeable via a wired cable -- phones, tablets, e-readers, earbuds, cameras, portable speakers, etc -- will have to be equipped with a port known as USB-C.\"\nQ: To help reduce cable clutter and waste, which continent will soon require Apple and other smartphone makers to support a single common charging standard for all mobile devicesin Bob's opinion?")


Response: Europe
Response: Asia


In [9]:
!pwd

/content


In [ ]:
evaluate(input("Instruction: "))